# ANNUITY CALCULATOR

In this notebook I use the simple mortgage calculator to generate mortgage cashflows.

In [2]:
from financepy.products.bonds.FinBondAnnuity import FinBondAnnuity
from financepy.finutils.FinCalendar import *
from financepy.finutils.FinDayCount import FinDayCountTypes
from financepy.finutils.FinFrequency import FinFrequencyTypes
from financepy.finutils.FinDate import FinDate
from financepy.products.libor.FinLiborDeposit import FinLiborDeposit
from financepy.products.libor.FinLiborSwap import FinLiborSwap
from financepy.market.curves.FinLiborCurve import FinLiborCurve
from financepy.finutils.FinMath import ONE_MILLION

####################################################################
TEMPORARY MESSAGE: Welcome to FinancePy!
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY and without even the implied warranty of 
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. Furthermore,
this is an alpha version of FinancePy & so is in early development.
Users are requested to verify outputs and if any doubt is found 
users are requested to contact the author of FinancePy. 
###################################################################



## CREATE AN ANNUITY STREAM

Calculate the cashflows on an annuity

In [3]:
settlementDate = FinDate(20, 6, 2018)

In [4]:
maturityDate = settlementDate.addTenor("5Y")  # FinDate(20, 9, 2028)
coupon = 0.05
frequencyType = FinFrequencyTypes.SEMI_ANNUAL
calendarType = FinCalendarTypes.WEEKEND
busDayAdjustType = FinBusDayAdjustTypes.FOLLOWING
dateGenRuleType = FinDateGenRuleTypes.BACKWARD
basisType = FinDayCountTypes.ACT_360
face = ONE_MILLION

In [5]:
annuity = FinBondAnnuity(maturityDate,coupon,frequencyType,calendarType,busDayAdjustType,dateGenRuleType,basisType,face)

In [6]:
print(annuity)

MATURITY DATE: TUE 20 JUN 2023
FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
CALENDAR: FinCalendarTypes.WEEKEND
BUSDAYRULE: FinBusDayAdjustTypes.FOLLOWING
DATEGENRULE: FinDateGenRuleTypes.BACKWARD



In [7]:
annuity.printFlows(settlementDate)

THU 20 DEC 2018 , 25416.666666666668
THU 20 JUN 2019 , 25277.777777777777
FRI 20 DEC 2019 , 25416.666666666668
SAT 20 JUN 2020 , 25416.666666666668
SUN 20 DEC 2020 , 25416.666666666668
SUN 20 JUN 2021 , 25277.777777777777
MON 20 DEC 2021 , 25416.666666666668
MON 20 JUN 2022 , 25277.777777777777
TUE 20 DEC 2022 , 25416.666666666668
TUE 20 JUN 2023 , 25277.777777777777


## We can value the flows by discounting them

First we build a Libor swap curve

In [8]:
depos = []
dcType = FinDayCountTypes.ACT_360
fixedFreq = FinFrequencyTypes.SEMI_ANNUAL
swap1 = FinLiborSwap(settlementDate,"1Y",0.0500,fixedFreq,dcType)
swap2 = FinLiborSwap(settlementDate,"2Y",0.0500,fixedFreq,dcType)
swap3 = FinLiborSwap(settlementDate,"3Y",0.0500,fixedFreq,dcType)
swap4 = FinLiborSwap(settlementDate,"4Y",0.0500,fixedFreq,dcType)
swap5 = FinLiborSwap(settlementDate,"5Y",0.0500,fixedFreq,dcType)
swaps = [swap1, swap2, swap3, swap4, swap5]

liborCurve = FinLiborCurve("USD_LIBOR", settlementDate, depos, [], swaps)

Here are the flows on the fixed leg of the 5Y swap

In [9]:
swap5.printFixedLeg()

START DATE: WED 20 JUN 2018
MATURITY DATE: TUE 20 JUN 2023
COUPON (%): 5.0
FIXED LEG FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FIXED LEG DAY COUNT: FinDayCountTypes.ACT_360
VALUATION DATE WED 20 JUN 2018
PAYMENT_DATE     YEAR_FRAC        FLOW         DF         DF*FLOW       CUM_PV
THU 20 DEC 2018  0.5083333         2.54   0.97521251         2.48         2.48
THU 20 JUN 2019  0.5055556         2.53   0.95116989         2.40         4.88
FRI 20 DEC 2019  0.5083333         2.54   0.92759512         2.36         7.24
MON 22 JUN 2020  0.5138889         2.57   0.90435657         2.32         9.56
MON 21 DEC 2020  0.5055556         2.53   0.88206005         2.23        11.79
MON 21 JUN 2021  0.5055556         2.53   0.86031325         2.17        13.97
MON 20 DEC 2021  0.5055556         2.53   0.83910260         2.12        16.09
MON 20 JUN 2022  0.5055556         2.53   0.81841489         2.07        18.16
TUE 20 DEC 2022  0.5083333         2.54   0.79812844         2.03        20.19
TUE 20 

In [10]:
cleanValue = annuity.cleanPriceFromDiscountCurve(settlementDate, liborCurve)

In [11]:
dirtyValue = annuity.fullPriceFromDiscountCurve(settlementDate, liborCurve)

The price of the annuity is quoted based on a par notional

In [12]:
print("CLEAN PRICE:", cleanValue)
print("DIRTY PRICE:", dirtyValue)
print("ACCRD VALUE:", annuity._accruedInterest)

CLEAN PRICE: 22.154860705530105
DIRTY PRICE: 22.154860705530105
ACCRD VALUE: 0.0


Copyright (c) 2020 Dominic O'Kane